In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# MUST MOUNT GOOGLE DRIVE FIRST OR THE CODE WILL NOT RUN
# These are all L2 data products, may add L1 to fill holes

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
import xarray as xr
import pandas as pd
import re, time, random 

data = '''Date          pressure    temperature          density
9/12/2014 9:00     170            4.0                1028
9/12/2014 10:00    368            4.2                1028.5
9/12/2014 11:00    368            4.2                1028.5'''
da = [[i for i in re.split("[ ][ ]+", l)] for l in data.split("\n")]
df2 = pd.DataFrame(da[1:], columns=da[0])
data='''Date        pressure      temperature           density
9/12/2014 9:00  177.859887      4.574663842         1028.477
9/12/2014 9:01  214.3598333     4.397781667         1028.66
9/12/2014 9:55  264.5863333     4.208137222         1028.905
9/12/2014 10:01  314.3161111     4.1242              1029.143
9/12/2014 10:02  363.8005587     4.02983352          1029.377'''
#pd.read_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/Apex_Array/Profiler Mooring/CTD_CSVs/Q1_14_17_3min_Depth_Profiler_Mooring.csv', sep=",")

da = [[i for i in re.split("[ ][ ]+", l)] for l in data.split("\n")]
df1 = pd.DataFrame(da[1:], columns=da[0])

#df1 = pd.read_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/Apex_Array/Profiler Mooring/CTD_CSVs/Q1_14_17_3min_Depth_Profiler_Mooring.csv', sep=",")
#df2 = pd.read_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/Apex_Array/Profiler Mooring/', sep=",")

df1.index = pd.to_datetime(df1.Date, format="%d/%m/%Y %H:%M", utc=True)
df2.index = pd.to_datetime(df2.Date, format="%d/%m/%Y %H:%M", utc=True)
df3 = df1.join(df2.resample("1min").pad(), rsuffix="_hourly")
df3

,Date,pressure,temperature,density,Date_hourly,pressure_hourly,temperature_hourly,density_hourly
Date,,,,,,,,
2014-12-09 09:00:00+00:00,9/12/2014 9:00,177.859887,4.574663842,1028.477,9/12/2014 9:00,170,4.0,1028
2014-12-09 09:01:00+00:00,9/12/2014 9:01,214.3598333,4.397781667,1028.66,9/12/2014 9:00,170,4.0,1028
2014-12-09 09:55:00+00:00,9/12/2014 9:55,264.5863333,4.208137222,1028.905,9/12/2014 9:00,170,4.0,1028
2014-12-09 10:01:00+00:00,9/12/2014 10:01,314.3161111,4.1242,1029.143,9/12/2014 10:00,368,4.2,1028.5
2014-12-09 10:02:00+00:00,9/12/2014 10:02,363.8005587,4.02983352,1029.377,9/12/2014 10:00,368,4.2,1028.5


In [ ]:
pip install netCDF4

     |████████████████████████████████| 4.3MB 2.8MB/s 
     |████████████████████████████████| 296kB 24.7MB/s 


In [ ]:
#Conversion script

import os
import xarray as xr
import pandas as pd

os.chdir("/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/SMAP_Data") #change last directory
#Point it to a directory of choice
for file_name in os.listdir("/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/SMAP_Data"): #change last directory
  if file_name.endswith(".nc"):
  # For loop for .nc files in the directory
   ds = xr.open_dataset(file_name)
   ds.data_vars
   df = ds.to_dataframe()
   
   #convert to a dataframe
   df.to_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/SMAP_Data/{}.csv'.format(file_name.split('.')[0])) #change last directory
   #daily arvage script to convert NetCDF to csv files
   #need to change the directory every time

In [ ]:
#Special edited version above for profiler mooring data to have data for MLD
import os
import xarray as xr
import pandas as pd

os.chdir("/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/Apex_Array/Profiler Mooring/CTD") #change last directory
#Point it to a directory of choice
for file_name in os.listdir("/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/Apex_Array/Profiler Mooring/CTD"): #change last directory
  if file_name.endswith(".nc"):
  # For loop for .nc files in the directory
   ds = xr.open_dataset(file_name)
   ds.data_vars
   df = ds.to_dataframe()
   
df.resample('D').mean().to_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/Apex_Array/Profiler Mooring/CTD/{}.csv'.format(file_name.split('.')[0])) #change last directory
#T is minutely 15T is every 15 mins D daily avaraging

In [ ]:
#combine script

import glob

os.chdir("/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/SMAP_Data") #change last directory
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/SMAP_Data/SMAP_Data.csv", index=False) #change last directory
#script that combines csv files to one file

In [ ]:
#print variables to final data base
import os
import pandas as pd
#RUN THIS AFTER MAKING ALL THE CSV FILES
finaldf = pd.DataFrame()
directory = "/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/Apex_Array/Flanking_Subsurface_A/CTD_CSVs"  #change last directory

for filename in os.listdir(directory):
    fullpath = os.path.join(directory, filename)
    if os.path.isfile(fullpath) and fullpath.endswith(".csv"):
        dfchild = pd.read_csv(fullpath,usecols=[0,8])
        #define columns you want to explort
        dfmaster = dfchild

        #new line you need to edit the titles in the excle file yourself this prints columns side by side
        # So with the data I used there were instruments from 10 meters to 2400 meter depth 
        #this takes the columns of the converted CSV files in the output directory and places them side by side in order

        finaldf = pd.concat([finaldf, dfmaster],axis = 1)
    print(dfmaster.reset_index(drop=True))
    finaldf.to_csv("/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/Apex_Array/Flanking_Subsurface_A/CTD_CSVs/pressure_A_CTD_surface.csv", index=False)

             obs  ctdmo_seawater_pressure
0     2014-09-16              2741.257083
1     2014-09-17              2765.777949
2     2014-09-18              2765.773540
3     2014-09-19              2765.821056
4     2014-09-20              2765.907941
...          ...                      ...
1792  2019-08-06              2771.786520
1793  2019-08-07              2771.696255
1794  2019-08-08              2771.696583
1795  2019-08-09              2771.696536
1796  2019-08-10              2771.624759

[1797 rows x 2 columns]
             obs  ctdmo_seawater_pressure
0     2014-09-12                21.237242
1     2014-09-13                17.957062
2     2014-09-14                19.001927
3     2014-09-15                16.862922
4     2014-09-16                16.622470
...          ...                      ...
1806  2019-08-07                33.785582
1807  2019-08-08                33.823667
1808  2019-08-09                33.783115
1809  2019-08-10                33.788543
1810  201

In [ ]:
#Below is some pandas filtiering i did, nothing too important the conversion aspect from Netcdf to csv was the important contributions

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/Apex_Array/Profiler Mooring/CTD_CSVs/Depth_Profiler_Mooring.csv', sep=",")
df['Diff'] = df['ctdpf_ckl_seawater_pressure'] - df['ctdpf_ckl_seawater_pressure'].shift(1)
df2 = df[df['Diff'] < -0.1]
df3 = df[df['Diff'] >= 0]
select_cols = ['obs', 'lon', 'lat', 'ctdpf_ckl_seawater_pressure', 'ctdpf_ckl_seawater_temperature', 'practical_salinity', 'density']
df2 = df2[select_cols]
df3 = df3[select_cols]
df2.to_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/Apex_Array/Profiler Mooring/CTD_CSVs/upcast_Depth_Profiler_Mooring.csv', sep=',', index=False)
df3.to_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/Apex_Array/Profiler Mooring/CTD_CSVs/downcast_Depth_Profiler_Mooring.csv', sep=',', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/MLD/Argo_mixedlayers_all_12112019.csv', sep=",")
df = df[df['profiledate'] > 735599.615891218]
df.to_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/MLD/1_1_2014_Argo_mixedlayers_all_12112019.csv', sep=',', index=False)

In [ ]:
#import pandas as pd
#import numpy as np
#df = pd.read_csv('C:\\Users\\truet\OneDrive\Desktop\\014-015.txt',index_col=[2], parse_dates=[0],usecols=[0,1,2,3,4,5,6,7,8], sep=",")
#print (df.index.values.astype('<M8[h]'))

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/MLD/1_1_2014_Argo_mixedlayers_all_12112019.csv', sep=",")
df = df[df['profilelat'] >= 50]
df.to_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/MLD/1_1_2014_lat_50+_Argo_mixedlayers_all_12112019.csv', sep=',', index=False)

In [ ]:
import os
import xarray as xr
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/MLD/MLD_59.06_60.62_and_neg_37.605_neg_41.505_indiv_argo_floats_1.csv", parse_dates=['Date'], index_col='Date')
#agg = df.resample('M',how='mean')
df.resample('15T').ffill().to_csv('/content/drive/My Drive/Irminger_2020_Project_Colab_Notebooks/MLD/Daily_MLD_59.06_60.62_and_neg_37.605_neg_41.505_indiv_argo_floats.csv')
#T is minutely 15T is every 15 mins

ValueError: ignored